In [ ]:
##Final Code

from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv

# Create an instance of the Chrome driver
driver = webdriver.Chrome()

# Navigate to the website
driver.get("https://www.nike.com/w?q=shoes")

# Infinite scroll until the end of the webpage
height = driver.execute_script('return document.body.scrollHeight')
while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(10)  # Sleep to allow page to load; adjust as needed.
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height == height:
        break
    height = new_height

# Get the HTML content after scrolling
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close the browser
driver.quit()

# Find all product cards on the page
product_cards = soup.find_all('div', class_='product-card')

# Prepare the data list
products_data = []

# Loop through the product cards and extract information
for card in product_cards:
    product_data = {}
    product_data['Product Name'] = card.find('div', class_='product-card__title').text.strip() if card.find('div', class_='product-card__title') else 'N/A'
    
    # Extracting the type of shoes (Men's, Women's, etc.)
    shoe_type_element = card.find('div', class_='product-card__subtitle')
    product_data['Shoe Type'] = shoe_type_element.text.strip() if shoe_type_element else 'N/A'

    product_prices = card.find('div', class_='product-price__wrapper')
    if product_prices:
        original_price_element = product_prices.find('div', class_='product-price', attrs={'data-testid': 'product-price'})
        discounted_price_element = product_prices.find('div', class_='product-price', attrs={'data-testid': 'product-price-reduced'})
        
        product_data['Original Price'] = original_price_element.text.strip() if original_price_element else 'N/A'
        product_data['Discounted Price'] = discounted_price_element.text.strip() if discounted_price_element else 'N/A'
    else:
        product_data['Original Price'], product_data['Discounted Price'] = 'N/A', 'N/A'

    discount_percentage_element = card.find('div', class_='product-price__perc')
    product_data['Discount Percentage'] = discount_percentage_element.text.strip() if discount_percentage_element else 'N/A'

    products_data.append(product_data)

# Specify the CSV file name
csv_file = "nike_products.csv"

# Writing to CSV
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    # Make sure to include 'Shoe Type' in the fieldnames
    writer = csv.DictWriter(file, fieldnames=['Product Name', 'Shoe Type', 'Original Price', 'Discounted Price', 'Discount Percentage'])
    writer.writeheader()
    writer.writerows(products_data)

print(f'Data has been written to {csv_file}')


